In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install 'crewai[tools]'==0.28.8 --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install duckduckgo-search==5.3.0 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
import os
import re
from datetime import datetime

import pandas as pd
import requests
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from google.colab import userdata
from langchain.agents import load_tools
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


def format_response(response: str) -> str:
    entries = re.split(r"(?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]


os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

## Tools

In [2]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [27]:
response = search_tool.run("Bitcoin")
format_response(response)

["snippet: Bitcoin continued to shed its value as Hong Kong's Bitcoin and Ether ETFs failed to impress investors on their debut. Recently, six crypto-related ETFs were listed on the Hong Kong exchange. Bitcoin ETFs contributed $8., title: Bitcoin falls to $61,000 as Hong Kong crypto ETFs fail to impress investors, link: https://www.msn.com/en-us/money/markets/bitcoin-falls-to-61-000-as-hong-kong-crypto-etfs-fail-to-impress-investors/ar-AA1nVR7T, date: 2024-04-30T14:18:00+00:00, source: Quartz on MSN.com",
 "snippet: The bull market for bitcoin (CRYPTO: BTC) might have hit its peak, as suggested by Peter Brandt, the CEO of Factor LLC and a seasoned chart analyst., title: Peter Brandt Suggests Bitcoin Bull Run May Have Peaked: '...Already Reached A Price Consistent', link: https://markets.businessinsider.com/news/stocks/peter-brandt-suggests-bitcoin-bull-run-may-have-peaked-already-reached-a-price-consistent-1033306810, date: 2024-04-30T14:28:00+00:00, source: Business Insider",
 'snippe

In [3]:
human_tools = load_tools(["human"])

In [4]:
def get_daily_closing_prices(ticker) -> pd.DataFrame:
    url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={ticker}&market=USD&apikey={userdata.get("ALPHAVANTAGE_API_KEY")}'
    response = requests.get(url)
    data = response.json()
    price_data = data["Time Series (Digital Currency Daily)"]
    daily_close_prices = {
        date: prices["4. close"] for (date, prices) in price_data.items()
    }

    df = pd.DataFrame.from_dict(daily_close_prices, orient="index", columns=["price"])
    df.index = pd.to_datetime(df.index)
    df["price"] = pd.to_numeric(df["price"])

    return df

In [7]:
price_df = get_daily_closing_prices("BTC")
price_df.head()

,price
2024-04-30,63976.61
2024-04-29,63844.85
2024-04-28,63108.94
2024-04-27,63422.00
2024-04-26,63742.39


In [29]:
text_output = []
for date, row in price_df.head(10).iterrows():
    text_output.append(f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}")
formatted_text = "\n".join(text_output)
print(formatted_text)

2024-04-30 - 63976.61
2024-04-29 - 63844.85
2024-04-28 - 63108.94
2024-04-27 - 63422.00
2024-04-26 - 63742.39
2024-04-25 - 64484.44
2024-04-24 - 64255.78
2024-04-23 - 66414.16
2024-04-22 - 66859.17
2024-04-21 - 64952.92


In [5]:
@tool("price tool")
def cryptocurrency_price_tool(ticker_symbol: str) -> str:
    """Get daily closing price for a given cryptocurrency ticker symbol for the previous 60 days"""
    price_df = get_daily_closing_prices(ticker_symbol)
    text_output = []
    for date, row in price_df.head(60).iterrows():
        text_output.append(f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}")
    return "\n".join(text_output)

In [6]:
@tool("search tool")
def cryptocurrency_news_tool(ticker_symbol: str) -> str:
    """Get news for a given cryptocurrency ticker symbol"""
    return search_tool.run(ticker_symbol + " cryptocurrency")

## Llama 3 with Groq

In [7]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [33]:
%%time
system = "You are experienced Machine Learning & AI Engineer."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm
response = chain.invoke({"text": "How to increase inference speed for a 7B LLM?"})

CPU times: user 59.1 ms, sys: 4.01 ms, total: 63.2 ms
Wall time: 2.4 s


In [34]:
print(response.content)

The quest for speed!

Inference speed is crucial for large language models (LLMs) like 7B parameter models, which can be computationally expensive to deploy. Here are some strategies to increase inference speed for a 7B LLM:

1. **Model Pruning**: Remove redundant or unnecessary weights and connections to reduce the model's size and computational requirements. This can be done using techniques like magnitude-based pruning, L1 regularization, or iterative pruning.
2. **Knowledge Distillation**: Train a smaller, simpler model (the student) to mimic the behavior of the large model (the teacher). The student model can be deployed for inference, reducing computational requirements.
3. **Quantization**: Represent model weights and activations using fewer bits (e.g., int8 instead of float32). This reduces memory usage and can lead to faster inference. Techniques like post-training quantization or quantization-aware training can be used.
4. **Tensor Train Decomposition**: Decompose the model's

## Agents

In [8]:
customer_communicator = Agent(
    role="Senior cryptocurrency customer communicator",
    goal="Find which cryptocurrency the customer is interested in",
    backstory="""You're highly experienced in communicating about cryptocurrencies
    and blockchain technology with customers and their research needs""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=human_tools,
)

In [9]:
news_analyst = Agent(
    role="Cryptocurrency News Analyst",
    goal="""Get news for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency news.
    You have a complete understanding of macroeconomic factors, but you specialize
    into analyzing news.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_news_tool],
)

price_analyst = Agent(
    role="Cryptocurrency Price Analyst",
    goal="""Get historical prices for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency
    historical prices. You have a complete understanding of macroeconomic factors,
    but you specialize into technical analys based on historical prices.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_price_tool],
)

In [10]:
writer = Agent(
    role="Cryptocurrency Report Writer",
    goal="""Write 1 paragraph report of the cryptocurrency market.""",
    backstory="""
    You're widely accepted as the best cryptocurrency analyst that
    understands the market and have tracked every asset for more than 10 years. Your trends
    analysis are always extremely accurate.

    You're also master level analyst in the traditional markets and have deep understanding
    of human psychology. You understand macro factors and combine those multiple
    theories - e.g. cycle theory. You're able to hold multiple opininons when analysing anything.

    You understand news and historical prices, but you look at those with a
    healthy dose of skepticism. You also consider the source of news articles.

    Your most well developed talent is providing clear and concise summarization
    that explains very complex market topics in simple to understand terms.

    Some of your writing techniques include:

    - Creating a bullet list (executive summary) of the most importannt points
    - Distill complex analyses to their most important parts

    You writing transforms even dry and most technical texts into
    a pleasant and interesting read.""",
    llm=llm,
    verbose=True,
    max_iter=5,
    memory=True,
    allow_delegation=False,
)

## Tasks

In [11]:
get_cryptocurrency = Task(
    description=f"Ask which cryptocurrency the customer is interested in.",
    expected_output="""Cryptocurrency symbol that the human wants you to research e.g. BTC.""",
    agent=customer_communicator,
)

In [12]:
get_news_analysis = Task(
    description=f"""
    Use the search tool to get news for the cryptocurrency

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",
    expected_output="""Create 1 paragraph report for the cryptocurrency,
    along with a prediction for the future trend
    """,
    agent=news_analyst,
    context=[get_cryptocurrency],
)


get_price_analysis = Task(
    description=f"""
    Use the price tool to get historical prices

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",
    expected_output="""Create 1 paragraph summary for the cryptocurrency,
    along with a prediction for the future trend
    """,
    agent=price_analyst,
    context=[get_cryptocurrency],
)

In [13]:
write_report = Task(
    description=f"""Use the reports from the news analyst and the price analyst to
    create a report that summarizes the cryptocurrency""",
    expected_output="""1 paragraph report that summarizes the market and
    predicts the future prices (trend) for the cryptocurrency""",
    agent=writer,
    context=[get_news_analysis, get_price_analysis],
)

## Crew

In [14]:
crew = Crew(
    agents=[customer_communicator, price_analyst, news_analyst, writer],
    tasks=[get_cryptocurrency, get_news_analysis, get_price_analysis, write_report],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15,
)

In [15]:
results = crew.kickoff()

 [DEBUG]: == Working Agent: Senior cryptocurrency customer communicator
 [INFO]: == Starting Task: Ask which cryptocurrency the customer is interested in.


> Entering new CrewAgentExecutor chain...
Thought: I need to ask the customer which cryptocurrency they are interested in.

Action: human
Action Input: {"question": "Hello! I'm here to help you with your cryptocurrency research. Which cryptocurrency are you interested in?"}

Hello! I'm here to help you with your cryptocurrency research. Which cryptocurrency are you interested in?
Bitcoin
 

Bitcoin

Thought: I think I have the answer, but I want to confirm.

Action: human
Action Input: {"question": "Just to confirm, you're interested in Bitcoin, correct?"}

Just to confirm, you're interested in Bitcoin, correct?
Correct
 

Correct

Thought: I now know the final answer
Final Answer: BTC

> Finished chain.
 [DEBUG]: == [Senior cryptocurrency customer communicator] Task output: BTC


 [DEBUG]: == Working Agent: Cryptocurrency News Ana